## Load Libraries

In [1]:
import pandas as pd
import numpy as np
import datetime 

In [2]:
# for web:
from flask import Flask, render_template, redirect, url_for, request

## Get Data from Data Warehouse

In [3]:
# read data 
df = pd.read_csv('Front end dataset.csv', encoding='cp1252')

In [4]:
# drop column named 'B' from DataFrame
# 'flight_date', 'departure', 'dicao', 'd_timezone', 'flight_status'
droplist = ['Flight ID', 'Unnamed: 0']
df.drop(droplist, axis=1, inplace=True)

In [5]:
df.shape

(18582, 22)

In [9]:
df.head(1)

,Arrival Airport ID,Departure Airport ID,Airline,Arriving scheduled time,Arriving terminal,Arriving gate,baggage line,Arrival airport,Departure scheduled time,Departure gate,...,Departure airpoty,Flight Status,Weather observed time,Temperature,Rain,Humidity,Clouds,Elevation,Visibility,Wind
0,KABQ,KJFK,JetBlue Airways,2022-04-27T22:43:00+00:00,NaN,NaN,NaN,Albuquerque International,2022-04-27T19:30:00+00:00,NaN,...,John F Kennedy International,scheduled,2022-04-27 17:52:00+00:00,"celsius: 23, fahrenheit: 73",NaN,percent: 31,"base_feet_agl: 7000, base_meters_agl: 2134, co...","feet: 5315, meters: 1620","meters: 16,100, meters_float: 16100.0, miles: ...","degrees: 200, gust_kph: 39, gust_kts: 21.0, gu..."


## search

In [10]:
app = Flask(__name__)

@app.route("/", methods=['POST', 'GET'])
def home():
    if request.method =='POST':
        # print(request.form)
        date = request.form["date"]
        depart = request.form["departure"]
        arrival = request.form["arrival"]
        flight_status = request.form["flight_status"]
        airline = request.form["airline"]
        #airline = request.form["airline"]

        #if arrival is None or arrival == '':
            #return abort(404)
        
        #verify posts
        print(type(date))
        print('From '+depart+' to '+arrival)
        
        # create a temperary database for searching
        flights = []
        
        # filtering the database
        # ------------------------filter by date
        flights = df[df['Departure scheduled time'].str.contains(date)]
        print(flights.shape)
        # ------------------------filter by dicao
        flights = flights[flights['Departure Airport ID'] == 'K'+ depart.upper()]
        print(flights.shape)
        # ------------------------filter by aicao
        flights = flights[flights['Arrival Airport ID'] == 'K'+ arrival.upper()]
        print(flights.shape)
        # ------------------------filter by status
        if flight_status != 'none':
            flights = flights[flights['Flight Status'] == flight_status]
        print(flights.shape)
        # ------------------------filter by airline
        if airline != '':
            flights = flights[flights['Airline'].str.contain(airline)]
        
        # clear old index
        flights.index = np.arange(1, len(flights) + 1)
        count = flights.shape[0]
        
        # return the final result
        return render_template("table.html", 
                           tables=[flights.to_html(classes='flights', index = True)], 
                           titles=flights.columns.values)
    #else:
    return render_template("simple.html")


# http://127.0.0.1:50xx
if __name__ == '__main__':
   app.run(host='localhost', port = 5080)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://localhost:5080/ (Press CTRL+C to quit)
127.0.0.1 - - [28/Apr/2022 19:18:16] "GET / HTTP/1.1" 200 -
